In [3]:
from with_FEM_MassMatrix import *
from convergence import *
from Reference_Solvers import *
from Experimental_Order_Cvg import *
from plots import *
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt

In [4]:
alpha = 0.1
beta = 1.
NSamples =200
pList = [0.01, 0.02, 0.03, 0.04,0.05]
model ={'name': 'check', 'alpha': alpha, 'beta': beta}
NCoarse = np.array([4])
Nepsilon = np.array([128])
NFine = np.array([256])    # Number of "epsilon-blocks" on τ_ε mesh in each direction (1-D array: [x_ε, y_ε, z_ε] if 3D etc.)     # Number of "coarse-blocks" on τ_H mesh in each direction (1-D array: [x_H, y_H, z_H]) # Number of layers in the patch 
Neigen = 3
k =3
np.random.seed(123)
root = 'Data/Sample_mining_N200_Ppt01_pt1_5_K3/'


data = sio.loadmat(root + '_meanErr_H' + str(32) + '.mat')
for i in data:
    if '__' not in i and 'readme' not in i:
            np.savetxt((root + '_meanErr_H' + str(32) + '.csv'),data[i],delimiter=',')

FileNotFoundError: [Errno 2] No such file or directory: 'Data/Sample_mining_N200_Ppt01_pt1_5_K3/_meanErr_H32.mat'